<h1>Predictive Model LCSF1: Lung Cancer Survival Forecasting for 1 Year</h1>


<h4>Workflow</h4><br>
<p>Here we lay out our thought process and what we did, while building a predictive model for lung cancer survival.</p>
<p></p>
<p>First we assume, that we have a choice of including a selection of more and / or less frequent </p>
<p></p>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

frequencySelectionApplied = 0    # 0 = do not apply, 1 = do apply

#selecting the standart dataset with no pre-selections
data = pd.read_csv("data/synthetic_data_lung_cancer.csv")

#if frequency selection has been done:
if frequencySelectionApplied == 1:
    data = pd.read_csv("data/bestFrequencyFiltered_t-low-0.3_t-high-1.csv") # adjust this import to match the result from the frequency_selection_jan notebook

print(data.columns)
data.tail()

Index(['SUBJECT_ID', 'DEFINITION_ID', 'TIME'], dtype='object')


,SUBJECT_ID,DEFINITION_ID,TIME
560966,984,measurement_1141,0.027321
560967,984,observation_156,0.028739
560968,984,measurement_1140,0.030802
560969,984,measurement_1327,0.035081
560970,984,condition_459,0.038022


Here we checked some data qualities and gained a better understanding of the data.<br>
A significant amount of this work has unfortunately been lost, but there is the Visual_Exploratory.ipynb where some graphs are retained.

In [2]:
data.loc[data['DEFINITION_ID'].str.contains('condition_*')]
#all condition observations are removed due to frequency Filtering

,SUBJECT_ID,DEFINITION_ID,TIME
1,1,condition_1922,0.008643
2,1,condition_785,0.027792
5,1,condition_1561,0.057292
7,1,condition_2391,0.065500
8,1,condition_175,0.166536
...,...,...,...
560922,984,condition_747,0.006456
560923,984,condition_1767,0.007445
560931,984,condition_517,0.008754
560964,984,condition_1496,0.023802


As a result we discovered that the frequency of DEFENITION_IDs corresponded to the types of DEFENITION_IDs. <br>
This meant that frequency selection is not useful in reducing recurrent data and noise. <br>
Therefore we concluded that maybe the recurrance of types of DEFENITION_IDs may have to be adjusted to have a fair comparison on the influence over the target feature.

Going further we decided to stretch out the DEFENITION_ID into multiple features.<br>
Both options for processing the data deliver the same final table with the difference being that the data format 0 has a seperate column for the actual IDs of the features [condition,procedure,drug,observation,measurement].

In [3]:
dataFormat = 1 # 0 for seperate ID column 1 for not
df = data.copy()
if dataFormat == 0:
    label_encoder = LabelEncoder()
    df['DEFINITION_ID_encoded'] = label_encoder.fit_transform(df['DEFINITION_ID'])
    df['condition'] = df['DEFINITION_ID'].apply(lambda x: 'condition' in x)
    df['procedure'] = df['DEFINITION_ID'].apply(lambda x: 'procedure' in x)
    df['drug'] = df['DEFINITION_ID'].apply(lambda x: 'drug' in x)
    df['observation'] = df['DEFINITION_ID'].apply(lambda x: 'observation' in x)
    df['measurement'] = df['DEFINITION_ID'].apply(lambda x: 'measurement' in x)
    df.drop(columns=['DEFINITION_ID'], inplace=True)
    df['time_since_last'] = df.groupby('SUBJECT_ID')['TIME'].diff().fillna(0)
    columns_to_convert = ['condition', 'procedure', 'drug', 'observation', 'measurement']
    for column in columns_to_convert:
        df[column] = df[column].astype(int)
    time_threshold = 1 
    df['death'] = df['time_since_last'].apply(lambda x: 1 if x <= time_threshold else 0)
elif dataFormat == 1:
    # Splitting 'DEFINITION_ID' into multiple columns
    new_cols = df['DEFINITION_ID'].str.split('_', expand=True)
    # Naming the new columns
    new_cols.columns = ['CATEGORY', 'CATEGORY_ID']
    print("New unique categorie: "+new_cols['CATEGORY'].unique())
    # Pivoting the DataFrame
    pivot_new_cols = new_cols.pivot_table(index=data.index, columns='CATEGORY', values='CATEGORY_ID', aggfunc=lambda x: ', '.join(str(v) for v in x)).fillna('-1')
    # Concatenating the new columns to the original DataFrame
    df = pd.concat([df, pivot_new_cols], axis=1)
    # Dropping the original 'DEFINITION_ID' column
    df.drop(columns=['DEFINITION_ID'], inplace=True)
    # Filling the death column 
    df['death'] = df['death'].replace('None', '1')
    df['death'] = df['death'].replace('-1', '0')
    # Converting column to boolean dtype
    df['death'] = df['death'].astype(int)
    # Creating additional column
    df['time_since_last'] = df.groupby('SUBJECT_ID')['TIME'].diff().fillna(0)
    columns_to_convert = ['condition', 'measurement', 'procedure','drug','observation']
    df[columns_to_convert] = df[columns_to_convert].astype(int, errors='ignore')
else:
    ...

print(df.columns)
#df.loc[(df['condition']=='')&(df['drug']=='')&(df['measurement']=='')&(df['observation']=='')&(df['procedure']=='')]
[print("Unique values in feature "+x+": "+str(len(df[x].unique()))) for x in df.columns]

df.head()

['New unique categorie: drug' 'New unique categorie: condition'
 'New unique categorie: measurement' 'New unique categorie: observation'
 'New unique categorie: procedure' 'New unique categorie: death']
Index(['SUBJECT_ID', 'TIME', 'condition', 'death', 'drug', 'measurement',
       'observation', 'procedure', 'time_since_last'],
      dtype='object')
Unique values in feature SUBJECT_ID: 727
Unique values in feature TIME: 560956
Unique values in feature condition: 2400
Unique values in feature death: 2
Unique values in feature drug: 419
Unique values in feature measurement: 1333
Unique values in feature observation: 225
Unique values in feature procedure: 491
Unique values in feature time_since_last: 560198


,SUBJECT_ID,TIME,condition,death,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,0,217,-1,-1,-1,0.000000
1,1,0.008643,1922,0,-1,-1,-1,-1,0.003836
2,1,0.027792,785,0,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,0,49,-1,-1,-1,0.004722
4,1,0.056765,-1,0,-1,132,-1,-1,0.024250


Here we also included a inversion of the target feature, because we initially thought that in this way we can predict if someone will not die in one year. (This is of course wrong)

In [4]:
targetFeatureInversion = 1

if targetFeatureInversion == 1:
    df['death']=df['death'].map(lambda a:0 if a==1 else 1)
df.head()

,SUBJECT_ID,TIME,condition,death,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,1,217,-1,-1,-1,0.000000
1,1,0.008643,1922,1,-1,-1,-1,-1,0.003836
2,1,0.027792,785,1,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,1,49,-1,-1,-1,0.004722
4,1,0.056765,-1,1,-1,132,-1,-1,0.024250


Here we included the option to make all the categorical columns resulting from DEFINITION_ID one hot encoded. This of course means that we get around 54.000 columns. This usually blows any calculations, feature selections, fitting and so on out of proportion. Hence we advise not to use it. 

In [5]:
#dummies for categorical analysis

createDummies = 0 # 0 = no, 1 = yes

if createDummies ==1 and dataFormat==1:
    df = pd.get_dummies(df,columns=['condition','drug','measurement','observation','procedure'])
    
df.head()

,SUBJECT_ID,TIME,condition,death,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,1,217,-1,-1,-1,0.000000
1,1,0.008643,1922,1,-1,-1,-1,-1,0.003836
2,1,0.027792,785,1,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,1,49,-1,-1,-1,0.004722
4,1,0.056765,-1,1,-1,132,-1,-1,0.024250


Next we split the data into training and test data. This should be done along the SUBJECT_ID, because obviously a real world data set would only include the DEFINITION_IDs of one individual.

In [6]:
from sklearn.model_selection import train_test_split

# List of unique SUBJECT_IDs
unique_subject_ids = df['SUBJECT_ID'].unique()

# Split the unique_subject_ids into train and test IDs
train_ids, test_ids = train_test_split(unique_subject_ids, test_size=0.3, random_state=2)

# Filter the main DataFrame into train and test sets based on the selected IDs
train_data = df[df['SUBJECT_ID'].isin(train_ids)]
test_data = df[df['SUBJECT_ID'].isin(test_ids)]

# Separate the features (X) and target variable (y) for train and test sets
X_train = train_data.drop('death',axis=1)# Drop 'SUBJECT_ID' and target column
y_train = pd.Series(train_data.death)

X_test = test_data.drop('death',axis=1)# Drop 'SUBJECT_ID' and target column
y_test = pd.Series(test_data.death)

if createDummies ==1 and dataFormat==1: # Here we have to account for nonsensical values
    X_train = X_train.drop(['condition_-1','drug_-1','measurement_-1','observation_-1','procedure_-1'],axis=1)
    X_test = X_test.drop(['condition_-1','drug_-1','measurement_-1','observation_-1','procedure_-1'],axis=1)
#old way:
#X = df.drop('death',axis=1).drop(['condition_-1','drug_-1','measurement_-1','observation_-1','procedure_-1'],axis=1)
#y = pd.Series(df.death)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

print(y_train.head())
X_train.head()

0    1
1    1
2    1
3    1
4    1
Name: death, dtype: int64


,SUBJECT_ID,TIME,condition,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,217,-1,-1,-1,0.000000
1,1,0.008643,1922,-1,-1,-1,-1,0.003836
2,1,0.027792,785,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,49,-1,-1,-1,0.004722
4,1,0.056765,-1,-1,132,-1,-1,0.024250


After the creation of the training and test data we tried to apply a sampling strategy. This unfortunately did not work as we imagined. This is because we wanted to balance the training set on every unique patient that survives or dies. This is not implemented yet correctly.

In [7]:
#undersampling

from sklearn.utils import resample

useUnderSampeling = 0 # 0 = no, 1 = yes

if useUnderSampeling == 1:
    # Separate majority and minority classes
    df_majority = df[df.death==0]
    df_minority = df[df.death==1]
    
    # Downsample majority class
    df_majority_downsampled = resample(df_majority,
                                     replace=False,    # sample without replacement
                                     n_samples=len(df_minority), # to match minority class
                                     random_state=123) # reproducible results
    
    # Combine minority class with downsampled majority class
    df_balanced = pd.concat([df_majority_downsampled, df_minority])
    
    # Display new class counts
    print(df_balanced.death.value_counts())
    print([len(df_balanced.loc[df_balanced['death']==1]),len(df_balanced.loc[df_balanced['death']==0, 'SUBJECT_ID'].unique())])
    
    # Separate X_train and y_train after balancing
    X_train_balanced = df_balanced.drop('death', axis=1)
    y_train_balanced = df_balanced['death']
    X_train = X_train_balanced
    y_train = y_train_balanced
else:
    print("Nothing happened")

Nothing happened


This feature selection is only relevant for when the dummies for the DEFINITION_ID-categories are created. This is to improve the model performance for training. This unfortunately takes massive amounts of resources and is propably not even very meaningful, because the massive amount of categories results in alot of noise. 

In [8]:
#feature selection

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression  # Replace with an appropriate estimator
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.sparse import csr_matrix


if createDummies==1:

#X_train_sparse = csr_matrix(X_train)

    # Initialize your estimator 
    lr=LogisticRegression(penalty='l1',solver='liblinear')
    svc=SVC(kernel="linear")
    nb=GaussianNB()
    
    estimator = lr  #change to whatever works best
    
    # Initialize RFECV with the estimator and scoring method
    rfecv=RFECV(estimator=estimator, cv=5)
    kbest=SelectKBest(score_func=f_classif, k=1000)
    selector = kbest
    
    # Fit the selector on your training data
    X_train_selected = selector.fit_transform(X_train, y_train)
    # Get selected feature indices
    selected_feature_indices = selector.get_support(indices=True)
    
    # Get the names of selected features
    selected_feature_names = X_train.columns[selected_feature_indices]
    
    # Create DataFrame with selected features
    X_train_selected_df = X_train[selected_feature_names]
    
    print(X_train_selected_df.head())
else:
    print("No feature selection, because of low feature count")

No feature selection, because of low feature count


In [9]:
# no scaling necessary

Here we apply preprocessing concerning dimensionality reduction techniques. We define a function which transfroms X_train and X_test into the desired format. <br>
We tried PCA and MCA. For MCA we had to apply the dimensionality reduction on only the categorical values and afterwards join them back in. The amount of dimensions is freely choosable. On these you can even apply regression models.

In [10]:
from prince import MCA
from sklearn.decomposition import PCA
#pip install prince

#applying MCA,PCA or others

def applyDimensionalityReduction(reductionTargetTrain,reductionTargetTest,typeOfReduction,n_components):
    if typeOfReduction == 'mca':
        X_train_categorical = reductionTargetTrain.drop(columns=['TIME','time_since_last'], inplace=False)
        X_test_categorical = reductionTargetTest.drop(columns=['TIME','time_since_last'], inplace=False)
        reducer = MCA(n_components=n_components)
        reducer.fit(X_train_categorical)
        transformed_train = reducer.transform(X_train_categorical)
        transformed_test = reducer.transform(X_test_categorical)
        X_train_mca = pd.concat([reductionTargetTrain[['TIME','time_since_last']],transformed_train],axis=1)
        X_test_mca = pd.concat([reductionTargetTest[['TIME','time_since_last']],transformed_test],axis=1)
        return X_train_mca,X_test_mca
    elif typeOfReduction =='pca':
        reductionTargetTrain.columns = reductionTargetTrain.columns.astype(str)
        reductionTargetTest.columns = reductionTargetTest.columns.astype(str)
        reducer = PCA(n_components=n_components)
        reducer.fit(reductionTargetTrain)
        transformed_train = reducer.transform(reductionTargetTrain)
        transformed_test = reducer.transform(reductionTargetTest)
        return pd.DataFrame(transformed_train), pd.DataFrame(transformed_test)
    else:
        return X_train,X_test

In [12]:
reduction_type = 'mca' # choose 'mca','pca' or 'None'

X_train_reduced,X_test_reduced = applyDimensionalityReduction(X_train,X_test,reduction_type,2) 

if 1:
    X_train = X_train_reduced
    X_test = X_test_reduced
X_train.head()

MemoryError: Unable to allocate 16.3 GiB for an array with shape (2191058688,) and data type float64

In [ ]:
#!pip install lightgbm
#!pip install xgboost

Here is the most important part of the workflow. The models are instatiated and selected for fitting.<br>
For some models we have hyperparameter tuning implemented. <br>


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


gSearch = 0 # select 0 or 1 if it should be executed or not (takes about 5 minutes)
grid_search = None
#Grid searches:
if gSearch == 1: 
    #grid search for XGB
    xgb_model = XGBClassifier(random_state=42)
    param_grid = {
        'max_depth': [3, 4, 5],
        'learning_rate': [0.1, 0.01, 0.05],
        'n_estimators': [100, 200, 300],
        'subsample': [0.8, 0.9, 1],
        'colsample_bytree': [0.3, 0.7, 1]
    }
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='roc_auc', cv=3, verbose=2)
    grid_search.fit(X_train, y_train)
else:
    ...

# model
log_reg = LogisticRegression(C=1.0, solver='liblinear', random_state=42)
lgbm = lgb.LGBMClassifier(random_state=42)
rf = RandomForestClassifier(n_estimators=50,criterion="gini",max_features=100,max_depth=4)
ada_boost_classifier = AdaBoostClassifier(base_estimator=rf, n_estimators=50, random_state=42)
svc = SVC(kernel="linear", probability=True)
xgb = XGBClassifier(random_state=42) if gSearch == 0 else grid_search.best_estimator_

all_models=[log_reg,lgbm,rf,ada_boost_classifier,svc,xgb]

if(reduction_type == 'mca'or reduction_type == 'pca'):
    print("Regression models are applicable.")

model = svc # select desired model

#fit model
model.fit(X_train,y_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Get feature importances
feature_importances = model.feature_importances_
feature_names = X_train.columns

# Sort feature importances in descending order
indices = np.argsort(feature_importances)[::-1]

# Limit the number of features to display
top_n = len(X_train.columns) if len(X_train.columns)<=100 else 100
indices = indices[:top_n]
feature_importances = feature_importances[indices]
feature_names = feature_names[indices]

# Plot feature importances
plt.figure(figsize=(16, 6))
plt.title("Top "+str(top_n)+" Feature Importances")
plt.bar(range(top_n), feature_importances, align="center")
plt.xticks(range(top_n), feature_names, rotation=90)
plt.xlabel("Feature Names")
plt.ylabel("Feature Importance")
plt.yscale('log')
plt.tight_layout()
plt.show()

Here we implemented cross validation.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

cv = KFold(n_splits=5)#, shuffle=True, random_state=42

#cross validation
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
print(scores)

In [ ]:
#predict probabilities
probs = model.predict_proba(X_test)
probs = pd.DataFrame(probs)
y_pred_proba = pd.Series(probs.iloc[:, 1])

# Calculate AUC
auc = roc_auc_score(y_test.values, y_pred_proba)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

threshold = 0.5
y_pred = np.where(y_pred_proba >= threshold, 1, 0)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"ROC AUC Score: {auc:.2f}")
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('graphs/linearSVC_nofrequency_noSampling_MCA.png', format='png')
plt.show()

Here is a iterative generation of images with the AUC for different models.<br>
Not yet implemented!

Tools used:<br>
- ChatGPT (this is due to our inexperience with complex python data processing, model training and plotting)<br>

Sources used: